In [36]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from openpyxl import Workbook,load_workbook
from time import sleep
from random import randint
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
import winsound
from selenium.webdriver.common.by import By
import win32com.client as win32
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

In [59]:
#leer el forms y obtener los inputs
scope = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:/Users/mario.vidana/actfin2-1650a1574146.json",scopes = scope)
file = gspread.authorize(creds)
workbook = file.open("Consulta_actfin").sheet1
rows = workbook.get_all_values()
df_forms = pd.DataFrame(rows)
df_forms.columns = df_forms.iloc[0]
df_forms.drop(df_forms.index[0], inplace = True)
df_forms = df_forms.rename(columns={df_forms.columns[0]: "Hora_consulta", df_forms.columns[1]: "Correo",df_forms.columns[2]: "Renta_venta",df_forms.columns[3]: "Estado",df_forms.columns[4]: "Municipio",df_forms.columns[5]: "dólar",df_forms.columns[6]: "Inmueble"})
df_forms

,Hora_consulta,Correo,Renta_venta,Estado,Municipio,dólar,Inmueble
1,10/10/2022 21:01:02,jaramario19@gmail.com,Bodegas,San Nicolas de los Garza Nuevo Leon,55,20,Bodegas
2,10/10/2022 23:29:31,mario.vidana@multimedios.com,Locales,Guadalupe Nuevo Leon,80,20,Locales
3,12/10/2022 18:26:26,jaramario19@gmail.com,Terreno,Guadalupe Nuevo Leon,70,20,Terreno
4,14/10/2022 16:41:55,mariovida99@gmail.com,Terreno,Saltillo Coahuila,75,20,Terreno
5,14/10/2022 16:59:50,mariovida99@gmail.com,Bodegas,san nicolas de los garza nuevo leon,50,20,Bodegas
6,14/10/2022 17:00:25,mariovida99@gmail.com,Locales,obispado nuevo leon,115,20,Locales
7,14/10/2022 17:21:18,mariovida99@gmail.com,Terreno,durango durango,55,20,Terreno
8,14/10/2022 17:22:35,mariovida99@gmail.com,Terreno,general escobedo nuevo leon,88,20,Terreno
9,14/10/2022 17:30:30,mariovida99@gmail.com,Terreno,saltillo coahuila,100,20,Terreno
10,14/10/2022 17:56:52,Jaramario19@gmail.com,Locales,"Guadalupe,Nuevo Leon",70,20,Locales


In [60]:
#leer la ultima consulta
f = open('C:/Users/mario.vidana/ultimo_indice.txt', 'r')
if f.mode=='r':
    indice= int(f.read())
    f.close()
f = open("C:/Users/mario.vidana/ultima_hora.txt","r")
if f.mode =="r":
    last_consult = f.read()
    last_consult_1 = datetime.strptime(last_consult,'%d/%m/%Y %H:%M:%S')
    f.close()

In [61]:
if datetime.strptime(df_forms.iloc[-1,0],'%d/%m/%Y %H:%M:%S') > last_consult_1:
    indice = indice + 1
    last_consult = df_forms.iloc[indice,0]
else:
    indice = indice
    last_consult = last_consult

file_indice = open("C:/Users/mario.vidana/ultimo_indice.txt", "w")
 
#convert variable to string

file_indice.write(str(indice))
 
#close file
file_indice.close()

file_hora = open("C:/Users/mario.vidana/ultima_hora.txt", "w")
 
#convert variable to string
file_hora.write(last_consult)
 
#close file
file_hora.close()

In [71]:
#inputs
if datetime.strptime(df_forms.iloc[-1,0],'%d/%m/%Y %H:%M:%S') > last_consult_1:
    duration = 1000
    freq = 440
    USD_MEX = round(float(df_forms.iloc[indice,5]),2)
    opciones = [df_forms.iloc[indice,6],df_forms.iloc[indice,2]]
    location = str(df_forms.iloc[indice,3]) + "/" + str(df_forms.iloc[indice,4])
    filtro_metros = 1000
    correo_solicitud = df_forms.iloc[indice,1]

    #Codigo
    path_doc = "C:/Users/mario.vidana/Documents/Python/Archivos_Descargas"
    location = location.replace(" ", "-")
    url = "https://inmuebles.metroscubicos.com/" + opciones[0] + "/" + opciones[1] + "/" + location
    browser = webdriver.Chrome("C:/Users/mario.vidana/Documents/Python/chromedriver.exe")
    browser.get(url)
    sleep(3)
    busquedas = browser.find_element(By.XPATH,"//span[@class = 'ui-search-search-result__quantity-results shops-custom-secondary-font']").text
    browser.close()
    busquedas = busquedas.replace(",","")
    busquedas = round(int(busquedas.split(" ")[0]))
    print(busquedas,url)

    #listas
    metros = []
    precios = []
    ubicacion = []
    links = []
    currency = []

    #extraer todos los resultados de la pagina
    pages = np.arange(1,busquedas+1,48)
    con = 1
    for page in pages:
        page = requests.get(url + '_Desde_' + str(page),verify = False)
        soup = BeautifulSoup(page.text, 'html.parser')
        mts = soup.find_all('li',class_='ui-search-card-attributes__attribute')
        ub = soup.find_all('span',class_='ui-search-item__group__element ui-search-item__location shops__items-group-details')
        pr = soup.find_all("span",class_="price-tag-fraction")
        ur = soup.find_all("a", class_="ui-search-result__content ui-search-link")
        sign = soup.find_all("span", class_="price-tag-symbol")
        sleep(randint(2,10))
        for i in ub:
            ubicacion.append(i.text)
        for i in mts:
            metros.append(i.text)
            metros = pd.Series(metros)
            searchfor = ['terreno', 'construidos']
            metros = metros[metros.str.contains('|'.join(searchfor))]
            metros = list(metros)
        for i in pr:
            precios.append(i.text)
        for i in ur:
            links.append(i.get('href'))
        for i in sign:
            currency.append(i.text)
            
        if len(metros) != len(precios):
            del metros[con:con+49]
            del precios[con:con+49]
            del ubicacion[con:con+49]
            del currency[con:con+49]
            del links[con:con+49]
        con = len(metros)
    
    winsound.Beep(freq,duration)

    

    #preproceso de datos
    #meter todo a un df
    df= pd.DataFrame({"Ubicacion":ubicacion,"Precio":precios,"Metros":metros,"Moneda":currency,"Liga":links})
    #limpieza y formato de datos y calcular el precio / m2
    df["split_metros"] = df.Metros.str.split(" ")
    df["mts2"] = df.split_metros.str.get(0)
    df["mts2"] = df.mts2.str.replace(",", "").astype(float)
    df["Precio"] = df.Precio.str.replace(",","").astype(float)

    #conversion a MXN
    df['Precio'] = np.where(df["Moneda"] == "U$S" , df.Precio*USD_MEX,df["Precio"])

    #mas limpieza
    df["Precio/m2"] = round(df.Precio / df.mts2,2)

    df = pd.DataFrame({"Ubicacion":df.Ubicacion.values,"Precio":df.Precio.values,"Metros":df.mts2.values,"Precio/m2":df["Precio/m2"],"Liga":df.Liga.values})

    #mandar info a un excel
    toexcel = pd.ExcelWriter(path_doc + "/Comparables_sin_filtro.xlsx")
    df.to_excel(toexcel,sheet_name="Sin filtros")
    toexcel.save()


    #filtros
    df.drop(df[df["Precio/m2"] <= 0.20].index, inplace = True)
    df.drop(df[df["Metros"] < filtro_metros].index, inplace = True)
    df.drop_duplicates(subset = ["Precio","Metros"],inplace = True)
    df.dropna(subset= ["Precio", "Metros","Precio/m2"])
    df = df.sort_values(by = ["Precio/m2"])
    df.reset_index(drop=True,inplace = True)

    #mandar info a otro excel
    toexcel = pd.ExcelWriter(path_doc + "/Comparables_con_filtro.xlsx")
    df.to_excel(toexcel,sheet_name="Con filtros")
    toexcel.save()

    minimo = min(np.array(df["Precio/m2"]))
    primer_c = np.percentile(np.array(df["Precio/m2"]),25)
    mediana = np.percentile(np.array(df["Precio/m2"]),50)
    tercer_c = np.percentile(np.array(df["Precio/m2"]),75)
    maximo = max(np.array(df["Precio/m2"]))
    print("minimo = " + str(minimo)+ "\nPrimer cuartil = " + str(primer_c) + "\nMediana = " + str(mediana) + "\nTercer cuartil = " + str(tercer_c)) 


    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.to = correo_solicitud
    mail.Subject = 'Consulta de comparables de ' + str(df_forms.iloc[indice,6]) + " en: " + str(df_forms.iloc[indice,4]) + " " + str(df_forms.iloc[indice,3])
    mail.HTMLBody = '<h3>A continuación encontrará la busqueda solicitada:</h3>'

    # To attach a file to the email (optional):
    attachment  = path_doc + "/Comparables_sin_filtro.xlsx"
    mail.Attachments.Add(attachment)
    attachment = path_doc + "/Comparables_con_filtro.xlsx"
    mail.Attachments.Add(attachment)

    mail.Send()
    
else:
    print("NO HAY SOLICITUDES NUEVAS")

C:\Users\mario.vidana\AppData\Local\Temp\ipykernel_13104\3606409228.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("C:/Users/mario.vidana/Documents/Python/chromedriver.exe")


101 https://inmuebles.metroscubicos.com/Bodegas/Renta/Nuevo-Leon/General-Escobedo


C:\Users\mario.vidana\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inmuebles.metroscubicos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\mario.vidana\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inmuebles.metroscubicos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\mario.vidana\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inmuebles.metroscubicos.com'. Adding certificate verification is strongly advised. See: https

minimo = 20.65
Primer cuartil = 70.0
Mediana = 80.0
Tercer cuartil = 94.4
